In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/historical_and_forecast_data.csv")

list=[]
for col in df.columns:
    if col not in ['index', 'Mar-25'] and not col.startswith('2'):
        list.append(col)

df_cleaned = df.drop(columns=list)

df_cleaned=df_cleaned.rename(columns={'Mar-25': 'Price_today'})

df_cleaned.head()


,index,Price_today,2025Q2,2025Q3,2025Q4,2026Q1,2026Q2,2026Q3,2026Q4,2027Q1,...,2027Q4,2028Q1,2028Q2,2028Q3,2028Q4,2029Q1,2029Q2,2029Q3,2029Q4,2030Q1
0,Albert Park-Middle Park-West St Kilda,700.0,697.421379,699.618152,694.810404,687.826767,685.415159,676.796165,679.935770,666.719495,...,664.976224,653.260323,657.406545,654.029519,663.603091,661.765671,669.900060,673.692795,682.941255,691.922189
1,Armadale,625.0,630.202314,637.680396,648.436551,652.752795,657.069039,661.385283,665.701527,670.017771,...,682.966502,687.282746,691.598990,695.915234,700.231478,704.547722,708.863966,713.180210,717.496454,721.812697
2,Carlton North,720.0,728.365576,735.993061,743.032473,749.603340,755.800894,761.701013,767.364150,772.838472,...,788.474907,793.507344,798.479158,803.402670,808.287699,813.142065,817.972001,822.782473,827.577436,832.360044
3,Carlton-Parkville,600.0,600.652519,601.556790,606.711298,601.026914,599.331589,599.695880,599.391098,599.260358,...,604.205223,608.435110,612.254466,615.899749,619.226922,623.789979,627.751477,631.401827,635.307710,638.824883
4,CBD-St Kilda Rd,650.0,642.940699,633.651440,621.726074,609.185029,596.816713,585.731950,576.569522,569.814935,...,564.710941,567.309664,571.333368,576.250730,581.535788,586.710022,591.373626,595.226278,598.076695,599.842110


In [8]:
df_avg = df_cleaned

df_avg["avg_future"]=df_avg.iloc[:,2:].mean(axis=1)


drop=[]

for col in df_avg.columns:
    if col not in ['index', 'Price_today', 'avg_future']:
        drop.append(col)

df_growth = df_avg.drop(columns=drop)

df_growth.head()

,index,Price_today,avg_future
0,Albert Park-Middle Park-West St Kilda,700.0,674.221781
1,Armadale,625.0,680.006297
2,Carlton North,720.0,784.110676
3,Carlton-Parkville,600.0,611.431314
4,CBD-St Kilda Rd,650.0,589.923889


In [18]:
df_growth['Price_change (%)'] = (df_growth['avg_future'] - df_growth['Price_today'])/df_growth['Price_today']*100

df_sorted = df_growth.sort_values(by='Price_change (%)', ascending=False)

df_sorted.head()

,index,Price_today,avg_future,Price_change (%)
130,Mildura,450.0,608.481381,35.218085
108,Seaford-Carrum Downs,550.0,726.785359,32.142793
96,Berwick,595.0,748.595230,25.814324
137,Shepparton,450.0,563.057993,25.123998
97,Cranbourne,580.0,715.200258,23.310389
